<h1 align="center"><font size="7"><strong>Final Project 2</strong></font></h1>

------------------------------------------------

# Kelompok 2 - PYTN KS 019
Ahmat Zainul Muttaqin

## 1. Overview (Perkenalan)
#### **Latar Belakang**
Keadaan alam, khususnya cuaca, memiliki dampak signifikan pada kehidupan manusia. Banyak aktivitas sehari-hari yang sangat bergantung pada kondisi cuaca, dan prakiraan cuaca menjadi penting untuk melaksanakan kegiatan dengan efisien. Beberapa faktor cuaca yang memengaruhi termasuk suhu, kelembaban, tekanan udara, kecepatan angin, dan lain sebagainya. Hujan menjadi fenomena penting yang dipengaruhi oleh faktor cuaca ini. Oleh karena itu, diperlukan analisis dan model prediktif untuk memproyeksikan kemungkinan hujan, memungkinkan masyarakat untuk merencanakan aktivitas mereka dengan lebih baik.

#### **Dataset**
Dataset yang digunakan untuk analisis ini adalah "Rain in Australia," diunduh dari Kaggle [di sini](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package).

Dataset ini berisi observasi harian prediksi hujan di seluruh Australia selama 10 tahun. Dataset ini terdiri dari 23 atribut dan 145.460 baris. Variabel targetnya adalah RainTomorrow, yang menunjukkan "No" (tidak hujan besok) dan "Yes" (hujan besok dengan 1mm atau lebih).

#### **Objective yang Ingin Dicapai**
Objective yang ingin dicapai dalam analisis ini yaitu:

- Dapat mengetahui faktor-faktor yang berpengaruh signifikan dalam memprediksi turunnya hujan

- Dapat memprediksi hujan tidaknya suatu tempat berdasarkan faktor-faktor yang mempengaruhinya menggunakan metode Regresi Logistik dan SVM

## 2. Import Pustaka yang Dibutuhkan

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import pickle

## 3. Data Loading
Bagian ini berisi proses data loading yang kemudian dilanjutkan dengan explorasi data secara sederhana.

a. Melakukan loading data dengan menggunakan library pandas

In [ ]:
df = pd.read_csv('weatherAUS.csv')

b. Melakukan setting untuk menampilkan semua kolom pada dataframe

In [ ]:
pd.set_option("display.max_columns", None)

c. Melihat 5 baris teratas pada dataset

In [ ]:
df.head()

d. Melihat 5 baris terakhir dari dataset

In [ ]:
df.tail()

e. melihat dimensi dataset

In [ ]:
df.shape

dapat dilihat dataset memiliki dimensi 23 kolom/atribut dan 145460 baris

f. melihat informasi (kolom, baris, tipe data) pada dataset

In [ ]:
df.info()

dapat dilihat dataset memiliki 57 atribut dengan tipe data float64 sebanyak 16, dan tipe data object sebanyak 7.

g. Melihat rincian nama kolom pada dataset

In [ ]:
df.columns

h. Mengecek apakah pada dataset terdapat missing value

In [ ]:
df.isnull().values.any()

output menunjukkan 'True' yang berarti pada dataset masih terdapat missing value

i. Melihat statistik deskriptif dari masing-masing atribut yang bertipe numeric

In [ ]:
df.describe()

j. Melihat statistik deskriptif dari masing-masing atribut yang bertipe object

In [ ]:
df.describe(include='O')

k. Melihat banyaknya unique values pada masing-masing atribut

In [ ]:
for col in df.columns:
    print(col,':', df[col].nunique())

l. Mencoba melihat unique values pada kolom Location

In [ ]:
df['Location'].unique()

In [ ]:
print(df['RainTomorrow'].value_counts())

## 4. Data Cleaning
Bagian ini mencakup langkah-langkah persiapan data sebelum melanjutkan eksplorasi lebih lanjut. Tahap pembersihan data melibatkan tindakan-tindakan seperti memberikan nama baru pada setiap kolom, menangani nilai yang hilang, menghilangkan kolom yang tidak relevan, dan langkah-langkah lain untuk memastikan kebersihan dan keteraturan data sebelum dilakukan analisis lebih lanjut.

a. Melihat apakah terdapat duplikasi data pada dataset

In [ ]:
df.duplicated().sum()

dapat dilihat tidak ada duplikasi data pada dataset

b. Melihat apakah terdapat _missing values_ / nilai yang hilang pada tiap kolom

In [ ]:
df.isnull().sum()

dari hasil diketahui terdapat missing value pada beberapa kolom dengan jumlah yang berbeda - beda

c. Melihat berapa persen missing value pada setiap kolom

In [ ]:
missing_value = df.apply(lambda x: f'{((x.isnull().sum()/df.shape[0])*100).round(2)} %')
missing_value

d. membuat visualisasi banyaknya missing value

In [ ]:
df.isnull().sum().plot.bar()
plt.show()

dapat dilihat kolom Sunshine, Evaporation, Cloud9am dan Cloud3am memiliki banyak missing value

e. Melakukan penghapusan kolom yang tidak diperlukan

disini kami hanya akan menggunakan kolom `Location`, `MaxTemp`, `MinTemp`, `Rainfall`, `Evaporation`, `Sunshine`, `WindGustSpeed`, `Humidity9am`, `Humidity3am`, `Cloud9am`, `Cloud3am`, `RainToday`, dan `RainTomorrow` untuk melakukan prediksi dan menghapus kolom selain itu

In [ ]:
drop_cols = ["Date", "WindGustDir", "WindDir9am", "WindDir3pm", "WindSpeed9am", "WindSpeed3pm", "Pressure9am", "Pressure3pm", 
             "Temp9am", "Temp3pm"]

Melakukan penghapusan atribut dengan .drop

In [ ]:
df = df.drop(drop_cols, axis=1)
df.head()

f. Mengisi missing values pada kolom numerical dan categorical
- menangani missing value pada atribut target (RainTomorrow) dengan melakukan penghapusan / drop baris

In [ ]:
df = df.drop(df[df['RainTomorrow'].isnull()].index)
df.shape

- pada kolom categorical ditangani dengan modus

In [ ]:
categorical = [i for i in df.columns if df[i].dtypes == 'O']

for cat_col in categorical:
    df[cat_col].fillna(df[cat_col].mode()[0],inplace=True)
    
df[categorical].isnull().sum()

- pada kolom numerical ditangani dengan median

In [ ]:
numerical = [i for i in df.columns if i not in categorical]

for num_col in numerical:
    df[num_col].fillna(df[num_col].median(),inplace=True)
    
df[numerical].isnull().sum()

In [ ]:
df.isnull().values.any()

setelah dilakukan penanganan missing value didapakan sudah tidak ada missing value lagi pada dataset

f. Memberikan nama baru pada untuk setiap kolom atau atribut

In [ ]:
df = df.rename(columns={
    'Location':'Lokasi',
    'MinTemp':'TempMin',
    'MaxTemp':'TempMax',
    'Rainfall':'CurahHujan', 
    'Evaporation':'Evaporasi',
    'Sunshine':'SinarMatahari',
    'WindGustSpeed':'KecepatanAngin',
    'Humidity9am':'Kelembapan9am',
    'Humidity3pm':'Kelembapan3pm',
    'Cloud9am':'Awan9am', 
    'Cloud3pm':'Awan3pm', 
    'RainToday':'HujanHariIni', 
    'RainTomorrow':'HujanBesok'})

df.head()

In [ ]:
df.columns

g. Melihat banyaknya nilai dari masing-masing atribut

In [ ]:
for col in df.columns:
    print(col, ': ')
    print(df[col].value_counts())
    print('\n','#'*60,'\n')

h. Membuat atribut atau kolom baru

disini kami membuat atribut atau kolom baru yaitu `SelisihTemp`, yang didapatkan dengan menghitung `TempMax` dikurang dengan `TempMin`

In [ ]:
df['SelisihTemp'] = df['TempMax'] - df['TempMin']

# melihat data setelah ditambahkan atribut / kolom baru
df.head()

## 5. Eksplorasi Data
Bagian ini berisi explorasi data pada dataset diatas dengan menggunakan query, grouping, visualisasi sederhana, dan lain sebagainya.

a. Mencari Curah hujan tertinggi dan terendah untuk lokasi Albury

In [ ]:
df.loc[df['Lokasi'] == 'Albury', 'CurahHujan'].agg(('min', 'max'))

Dari hasil dketahui untuk lokasi Albury curah hujan terendah yaitu 0.0 dan curah hujan tertinggi yaitu 104.2

b. Mencari waktu terlama dan tercepat hari cerah selama 24 jam. (`CahayaMatahari`)

In [ ]:
df.max(axis=0)['SinarMatahari']

In [ ]:
df.min(axis=0)['SinarMatahari']

Didapatkan waktu terlama sinar matahari muncul dalam 24 jam dengan lama **14.5 jam**, dan waktu tercepat dengan lama **0 jam**.

c. Mencari nilai tertinggi dan terendah pada kecepatan angin.

In [ ]:
df.max(axis=0)['KecepatanAngin']

In [ ]:
df.min(axis=0)['KecepatanAngin']

Didapatkan kecepatan angin tertinggi dengan nilai **135km/jam**. Sedangkan untuk kecepatan angin terendah didapat dengan nilai **0km/jam**.

d. Melihat proporsi data pada kolom Lokasi dan HujanHariIni dengan menggunakan crosstab

In [ ]:
pd.crosstab(df.HujanHariIni, df.Lokasi)

Pengamatan menunjukkan bahwa secara umum, sebagian besar lokasi cenderung jarang mengalami hujan setiap harinya. Portland, di sisi lain, menonjol karena perbedaan yang cukup tipis antara hari yang mengalami hujan dan tidak. Sebaliknya, lokasi Uluru cenderung jarang mengalami hujan.

e. Mengidentifikasi rata-rata (mean) curah hujan dengan melakukan grouping berdasarkan lokasi.

In [ ]:
df_mean = df[['Lokasi','CurahHujan']].groupby('Lokasi').mean().sort_values(by='CurahHujan', ascending=False)
df_mean.T

Dapat diamati bahwa daerah Cairns memiliki rata-rata curah hujan tertinggi dengan nilai sebesar 5.76 mm. Di sisi lain, Woomera mencatat nilai terendah dengan curah hujan rata-rata sebesar 0.48 mm.

f. Untuk melihat data histori top 5 record berdasarkan kriteria berikut:
- Lokasi berada di daerah Richmond.
- Hari di mana hujan turun.
- Kecepatan angin 30 km/jam atau lebih.

In [ ]:
df[
    (df['Lokasi'] == 'Richmond') &
    (df['HujanHariIni'] == 'Yes') &
    (df['KecepatanAngin'] >= 30) 
].head()

g. Melihat statistik deskriptif pada data numerik

In [ ]:
df.describe()

Dari hasil observasi, beberapa informasi kunci dapat diidentifikasi:

- Rata-rata curah hujan adalah 2.33 mm.
- Rata-rata tingkat evaporasi adalah 5.18.
- Rata-rata durasi sinar matahari adalah 8.04 jam.
- Suhu terendah dicapai pada kolom TempMin sebesar -8.50 derajat Celsius.
- Suhu tertinggi tercatat pada kolom TempMax dengan nilai 48.10 derajat Celsius.

Informasi ini memberikan gambaran umum tentang kondisi cuaca berdasarkan dataset yang digunakan.

h. Melihat statistik deskritif pada data kategorik

In [ ]:
df.describe(include="O")

dari hasil kita dapat melhat beberapa informasi seperti:
- Terdapat 49 lokasi dengan lokasi paling banyak terletak di Canberra
- Tidak hujan (No) lebih dominan pada atribut Hari_ini_Hujan dan Hari_besok_Hujan

i. Visualisasi line plot korelasi antara kecepatan angin dan dengan tinggi curah hujan.

In [ ]:
sns.lineplot(x='KecepatanAngin', y='CurahHujan', data=df)
plt.show()

Dapat diambil kesimpulan bahwa tinggi curah hujan cenderung berkorelasi dengan laju angin, meskipun tidak secara signifikan. Terdapat anomali pada nilai laju angin, terutama pada rentang 100-120 km/jam dan 120-140 km/jam. Perlu diperhatikan bahwa anomali ini dapat menjadi fokus lebih lanjut dalam analisis untuk memahami penyebab dan dampaknya pada kondisi cuaca.

j. Melihat persebaran data pada atribut HujanHariIni dan atribut target HujanBesok

In [ ]:
# Plot untuk 'HujanBesok'
plt.subplot(1, 2, 1)
df['HujanBesok'].value_counts().plot.bar(color='skyblue')
plt.title('Persebaran Hujan Besok')
plt.xlabel('Hujan Besok')
plt.ylabel('Jumlah')

# Plot untuk 'HujanHariIni'
plt.subplot(1, 2, 2)
df['HujanHariIni'].value_counts().plot.bar(color='purple')
plt.title('Persebaran Hujan Hari Ini')
plt.xlabel('Hujan Hari Ini')
plt.ylabel('Jumlah')

plt.tight_layout()
plt.show()


Terlihat kedua histogram menunjukkan proporsi yang sama. Pada atribut target HujanBesok, terdapat perbedaan yang cukup signifikan antara nilai No dan Yes

k. Melihat intensitas HujanBesok di setiap lokasi

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.countplot(x="Lokasi", hue="HujanBesok", data=df)
plt.title('Count Plot: Hujan Besok berdasarkan Lokasi')
plt.xlabel('Lokasi')
plt.ylabel('Jumlah')
plt.show()

Terlihat bahwa HujanBesok mempunyai intensitas yang berbeda-beda di setiap lokasi.

l. Visualisasi pie chart 8 lokasi dengan rata-rata curah hujan tertinggi

In [ ]:
df_mean_top8 = df_mean.head(8)
df_area = df_mean_top8.groupby(['Lokasi']).sum()['CurahHujan'].sort_values(ascending=False)
df_area.plot(kind='pie', figsize=(22,8), autopct='%1.0f%%')

plt.title("Pie Chart 8 Lokasi dengan Rata-Rata Curah Hujan Tertinggi")
plt.show()

dari hasil yang didapatkan dengan beberapa kecepatan angin yang ada histori terjadinya hujan hari ini di beberapa waktu hanya 24% dan tidak terjadinya hujan 76%

k. Visualisasi boxplot perbandingan HujanBesok berdasarkan TempMax

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))

sns.boxplot(x='HujanBesok', y='TempMax', data=df, palette="pastel")
ax.set_xticklabels(['Tidak Hujan', 'Hujan'])
plt.xlabel("Hujan Besok", fontsize=12)
plt.ylabel("Suhu Maksimum", fontsize=12)
plt.title("Hujan Besok vs Suhu Maksimum", fontsize=15)

plt.show()


## 6. Data Preprocessing
Bagian ini berisi proses penyiapan data untuk proses pelatihan model, seperti pembagian data menjadi train-dev-test, transformasi data (normalisasi, encoding, dll.), dan proses-proses lain yang dibutuhkan.

a. Melakukan inisialisasi variabel X dan y

Disini kami mendefinisikan untuk variabel X nya yaitu kolom selain `HujanBesok` dan `SelisihTemp`, sedangkan untuk variabel y yaitu `HujanBesok`

In [ ]:
X = df.drop(['HujanBesok', 'SelisihTemp'], axis=1)
y = df['HujanBesok']

print("Dimensi X:", X.shape)
print("Dimensi y:", y.shape)

b. Encoding atribut kategorik menggunakan replace function pada atribut HujanBesok ,HujanHariIni, dan Lokasi

In [ ]:
# Encoding pada atribut HujanBesok dan HujanHariIni

df['HujanHariIni'].replace({'No':0, 'Yes': 1}, inplace = True)
df['HujanBesok'].replace({'No':0, 'Yes': 1}, inplace = True)

In [ ]:
# Encoding pada atribut Lokasi
def encode_data(feature_name):
     
    mapping_dict = {}
    unique_values = list(df[feature_name].unique())
    for idx in range(len(unique_values)):
        mapping_dict[unique_values[idx]] = idx
    print(mapping_dict)
    return mapping_dict

df['Lokasi'].replace(encode_data('Lokasi'), inplace = True)

In [ ]:
# Melihat 5 baris teratas dataset yang telah dilakukan encoding
df.head()

c. Melakukan inisialisasi ulang variabel X dan y setelah dilakukan encoding

In [ ]:
X_transform = df.drop(['HujanBesok', 'SelisihTemp'], axis=1)
y = df['HujanBesok']

print(X_transform.shape)
print(y.shape)

d. Mengecek ulang korelasi

In [ ]:
plt.figure(figsize=(16, 9))
sns.heatmap(df.corr(), annot=True)
plt.show()

Dari hasil matriks korelasi, beberapa temuan menarik dapat diidentifikasi:

- Atribut seperti Lokasi, TempMax, Evaporasi, SinarMatahari, dan SelisihTemp menunjukkan korelasi negatif dengan atribut target HujanBesok. Ini dapat menunjukkan bahwa lokasi tertentu, suhu maksimum harian, evaporasi, durasi sinar matahari, dan selisih suhu memiliki pengaruh negatif terhadap kemungkinan hujan besok.

- Sebaliknya, atribut lainnya menunjukkan korelasi positif dengan HujanBesok, menunjukkan bahwa kelembaban pada pukul 3 sore (Kelembaban3pm), kondisi awan pada pukul 3 sore (Awan3pm), dan apakah hujan terjadi hari ini (HujanHariIni) memiliki pengaruh positif terhadap kemungkinan hujan besok.

- Terdapat atribut-atribut tertentu yang memiliki korelasi cukup tinggi dengan HujanBesok, yaitu Kelembaban3pm, Awan3pm, dan HujanHariIni. Korelasi yang tinggi antara atribut-atribut ini dan HujanBesok menunjukkan adanya hubungan kuat antara faktor-faktor ini dengan prediksi hujan besok.


e. Melakukan feature selection menggunakan library F regression dengan k=12

In [ ]:
topFeatures = SelectKBest(f_regression, k=12)
fit = topFeatures.fit(X_transform, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_transform.columns)
 
featureScores = pd.concat([dfcolumns, dfscores],axis=1)
featureScores.columns = ['Features', 'Score']

print("Top 12 Features yang Berpengaruh")
print(featureScores.nlargest(12, 'Score'))

dengan menggunakan library F regression juga didapatkan bahwa 12 feature yang paling berpengaruh dalam model yaitu `Kelembapan3pm`, `SinarMatahari`, `HujanHariIni`, `Awan3pm`, `Kelembapan9am`, `Awan9am`, `CurahHujan`, `KecepatanAngin`, `TempMax`, `Evaporasi`, `TempMin`, dan `Lokasi`

f. Memasukkan hasil dari feature seletion ke dalam variabel X_fit untuk selanjutnya dilakukan pemodelan

In [ ]:
X_fit = df[['Kelembapan3pm', 'SinarMatahari', 'HujanHariIni', 'Awan3pm', 'Kelembapan9am', 'Awan9am', 'CurahHujan', 
            'KecepatanAngin', 'TempMax', 'Evaporasi', 'TempMin', 'Lokasi']]

g. Melihat 5 baris teratas dataframe X_fit

In [ ]:
X_fit.head()

h. melihat informasi (kolom, baris, tipe data) dan dimensi pada dataframe X_fit

In [ ]:
X_fit.info()

X_fit.shape

i. Membagi data menjadi Training dan Testing pada kolom atau atribut yang sudah diperoleh. Untuk proporsi Training data sebesar 70% (0.70) dan Testing data sebesar 30% (0.30)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_fit, y, test_size=0.30, random_state=13)

In [ ]:
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)

Berdasarkan pembagian proporsi training data/data latih dan testing data/data uji, didapatkan jumlah **data training dari keseluruhan data sebanyak 99535 data**, serta **data testing sebanyak 42658 data**.

##  7. Pendefinisian Model
Pendefinisian Model Logistic Regression pada bagian ini dilakukan dengan menggunakan metode regresi logistik. Pemilihan metode ini didasarkan pada fakta bahwa regresi logistik cocok untuk masalah klasifikasi biner, seperti yang dihadapi dalam prediksi cuaca (hujan atau tidak hujan).

**a. Model Logistic Regression**

Alasan penggunaan metode ini mencakup kemampuannya menghitung probabilitas untuk setiap kelas, yang kemudian digunakan untuk membuat keputusan akhir dengan mengambil probabilitas tertinggi. Model Logistic Regression ini diharapkan dapat memberikan pemahaman yang baik tentang faktor-faktor yang mempengaruhi prediksi cuaca dan memberikan kinerja yang memadai dalam kasus klasifikasi biner ini.

In [ ]:
logreg = LogisticRegression(solver='liblinear', random_state=0)

**b. Model Support Vector Machines (SVM)**

Model pengujian kedua yakni model SVM. Model ini digunakan sebagai pembanding model logistic regression. Alasan menggunakan model SVM karena SVM mampu mengolah data berdimensi tinggi tanpa mengalami penurunan performa yang signifikan.

In [ ]:
svm_model = SVC(kernel='linear') #linear kernel

linear kernel dapat digunakan sebagai dot product pada dua pengamatan yang diberikan.

## 8. Pelatihan Model
Cell pada bagian ini hanya berisi code untuk melatih model dan output yang dihasilkan.

**a. Model Logistic Regression**

In [ ]:
logreg = logreg.fit(X_train, y_train)
logreg

**b. Model Support Vector Machines (SVM)**

In [ ]:
modelsvm = svm_model.fit(X_train, y_train)
modelsvm

## 9. Evaluasi Model
Pada bagian ini, dilakukan evaluasi model yang harus menunjukkan bagaimana performa model berdasarkan metrics yang dipilih. Hal ini harus dibuktikan dengan visualisasi tren performa dan/atau tingkat kesalahan model.

**a. Model Logistic Regression**
- memeriksa performa dengan .predict_proba() yang mengembalikan matriks probabilitas bahwa keluaran yang diprediksi sama dengan nol / satu

In [ ]:
logreg.predict_proba(X_test)

- melihat predicted outputs dengan .predict()

In [ ]:
y_pred_logreg = logreg.predict(X_test)
y_pred_logreg

- melihat nilai akurasi dengan .score

In [ ]:
print("Test score:", logreg.score(X_test, y_test))
print("Train score:", logreg.score(X_train, y_train))

didapatkan hasil nilai akurasi untus test data / pengujian data yaitu 84% (0.840), sedangkan jika dibandingkan dengan train data /pelatihan data nilai akurasinnnya 83% (0.835)

- Melihat confussion matrix dan classification report dari model

In [ ]:
confusion_matrix(y_test, y_pred_logreg)

In [ ]:
cm_logreg = confusion_matrix(y_test, y_pred_logreg)
sns.heatmap(cm_logreg, annot=True, fmt="d", linewidths=.5, cmap = 'Greys')
plt.show()
print(classification_report(y_test, y_pred_logreg))

berdasarkan hasil confusion matrix diketahui banyaknya nilai TP (True Positive) adalah 31388, FP (False Positive) adalah 1748, FN (False Negative) adalah 5041, dan TN (True Negative) adalah 4481

- Melihat summary dari evaluasi model Logistic Regression

In [ ]:
print('Ringkasan Evaluasi Model')
print('Akurasi :', accuracy_score(y_test, y_pred_logreg))
print('Presisi :', precision_score(y_test,y_pred_logreg))
print('Recall :', recall_score(y_test,y_pred_logreg))
print('F1 Score :', f1_score(y_test,y_pred_logreg))

**b. Model Support Vector Machines (SVM)**
- melihat predicted outputs dengan .predict()

In [ ]:
y_pred_svm = modelsvm.predict(X_test)
y_pred_svm

- melihat nilai akurasi dengan .score

In [ ]:
print("Test score:", modelsvm.score(X_test, y_test))
print("Train score:", modelsvm.score(X_train, y_train))

didapatkan hasil nilai akurasi untuk test data / pengujian data yaitu 84% (0.840), sedangkan jika dibandingkan dengan train data / pelatihan data nilai akurasinnnya 83% (0.834)

- Melihat confussion matrix dan classification report dari model

In [ ]:
confusion_matrix(y_test, y_pred_svm)

In [ ]:
cm_svm = confusion_matrix(y_test, y_pred_svm)
sns.heatmap(cm_svm, annot=True, fmt="d", linewidths=.5, cmap = 'twilight')
plt.show()
print(classification_report(y_test, y_pred_svm))

berdasarkan hasil confusion matrix diketahui banyaknya nilai TP (True Positive) adalah 31737, FP (False Positive) adalah 1399, FN (False Negative) adalah 5408, dan TN (True Negative) adalah 4114

- Melihat summary dari evaluasi model SVM

In [ ]:
print('Ringkasan Evaluasi Model')
print('Akurasi :', accuracy_score(y_test, y_pred_svm))
print('Presisi :', precision_score(y_test,y_pred_svm))
print('Recall :', recall_score(y_test,y_pred_svm))
print('F1 Score :', f1_score(y_test,y_pred_svm))

**Dari proses evaluasi 2 model kita dapat mengetahui bahwa:**
- Nilai akurasi dari model regresi logistik dan SVM hampir sama, keduanya mencapai akurasi sebesar 84%, yang menunjukkan bahwa keduanya dapat digunakan untuk melakukan prediksi hujan di Australia dengan baik.
- Model SVM menunjukkan kinerja lebih baik dalam hal recall dibandingkan dengan model logistic regression. Ini berarti model SVM lebih baik dalam mengidentifikasi kasus positif yang sebenarnya, yaitu ketika hujan benar-benar terjadi.
- Sebaliknya, model logistic regression memiliki nilai presisi dan F1 score yang lebih tinggi dibandingkan dengan model SVM. Hal ini menunjukkan bahwa model logistic regression lebih baik dalam meminimalkan jumlah kesalahan positif palsu, sehingga memberikan prediksi yang lebih akurat dalam mengidentifikasi kondisi hujan.

## 10. Model Inference
Dalam bagian ini, model inference dilakukan untuk menguji kinerja model pada data baru yang tidak termasuk dalam data train. Data yang digunakan untuk pengujian ini dapat berasal dari sumber eksternal seperti data buatan oleh pengguna atau data yang ditemukan di internet. Proses inferensi ini bertujuan untuk mengevaluasi sejauh mana model dapat memberikan prediksi yang akurat dan konsisten pada data baru yang belum pernah dilihat sebelumnya.

**Petunjuk Pengisian :**
- Silahkan masukkan lokasi yang ingin untuk dilakukan prediksi
- Untuk Hujan Hari Ini masukkan pilihan Yes atau No
- Untuk kolom lain silahkan masukkan sesuai dengan data yang ingin di prediksi dengan bertipe data float

In [ ]:
Lokasi = input("Lokasi : ")
TempMin = float(input("Suhu Minimal : "))
TempMax = float(input("Suhu Maksimal : "))
CurahHujan = float(input("Curah Hujan : "))
Evaporasi = float(input("Evaporasi : "))
SinarMatahari = float(input("Sinar Matahari : "))
KecepatanAngin = float(input("Kecepatan Angin : "))
Kelembapan9am = float(input("Kelembapan Pukul 9 am : "))
Kelembapan3pm = float(input("Kelembapan Pukul 3 pm : "))
Awan9am = float(input("Awan Pukul 9 am : "))
Awan3pm = float(input("Awan Pukul 3 pm : "))
HujanHariIni = input("Hari ini Hujan? (Yes/No) : ")

val = [TempMin, TempMax, CurahHujan, Evaporasi, SinarMatahari, KecepatanAngin, Kelembapan9am, Kelembapan3pm, Awan9am, Awan3pm]

loc = {'Albury': 0, 'BadgerysCreek': 1, 'Cobar': 2, 'CoffsHarbour': 3, 'Moree': 4, 'Newcastle': 5, 'NorahHead': 6, 
       'NorfolkIsland': 7, 'Penrith': 8, 'Richmond': 9, 'Sydney': 10, 'SydneyAirport': 11, 'WaggaWagga': 12, 'Williamtown': 13, 
       'Wollongong': 14, 'Canberra': 15, 'Tuggeranong': 16, 'MountGinini': 17, 'Ballarat': 18, 'Bendigo': 19, 'Sale': 20, 
       'MelbourneAirport': 21, 'Melbourne': 22, 'Mildura': 23, 'Nhil': 24, 'Portland': 25, 'Watsonia': 26, 'Dartmoor': 27, 
       'Brisbane': 28, 'Cairns': 29, 'GoldCoast': 30, 'Townsville': 31, 'Adelaide': 32, 'MountGambier': 33, 'Nuriootpa': 34, 
       'Woomera': 35, 'Albany': 36, 'Witchcliffe': 37, 'PearceRAAF': 38, 'PerthAirport': 39, 'Perth': 40, 'SalmonGums': 41, 
       'Walpole': 42, 'Hobart': 43, 'Launceston': 44, 'AliceSprings': 45, 'Darwin': 46, 'Katherine': 47, 'Uluru': 48}

for i in range(0,48):
    if loc[Lokasi]==i:
        val.append(1.0)
    else:
        next

hujan = {'No':0, 'Yes': 1}

for i in range(0,2):
    if hujan[HujanHariIni]==i:
        val.append(1.0)
    else:
        next
        
print(val)

predict_logreg = logreg.predict([val])
predict_svm = modelsvm.predict([val])

if predict_logreg == 0 : 
    print("Berdasarkan prediksi menggunakan Logistic regression Maka Besok tidak hujan")
else: 
    print("Berdasarkan prediksi menggunakan Logistic regression Maka Besok hujan")
    
if predict_svm == 0 :
    print("Berdasarkan prediksi menggunakan SVM Maka Maka Besok tidak hujan")
else: 
    print("Berdasarkan prediksi menggunakan SVM Maka Maka Besok hujan")

### Save and Finalize Model
Menggunakan operasi Pickle untuk menyimpan format serialized ke sebuah file dan menggunakannya untuk membuat prediksi baru

In [ ]:
pickle.dump(logreg, open('model_logreg_FP2.pkl', 'wb'))
pickle.dump(modelsvm, open('model_svm_FP2.pkl', 'wb'))

## 11. Pengambilan Kesimpulan

Dari analisis yang telah kami lakukan, kami dapat menyimpulkan beberapa poin kunci:

1. **Faktor-Faktor Berpengaruh:** Dari 23 atribut pada dataset, kami menemukan bahwa hanya 12 atribut yang menjadi faktor kuat yang paling mempengaruhi prediksi hujan di Australia. Atribut tersebut adalah `Lokasi`, `TempMin`, `TempMax`, `CurahHujan`, `Evaporasi`, `SinarMatahari`, `KecepatanAngin`, `Kelembapan9am`, `Kelembapan3pm`, `Awan9am`, `Awan3pm`, dan `HujanHariIni`.

2. **Penanganan Missing Value:** Kami mengatasi nilai yang hilang dengan mengisi missing value menggunakan median untuk atribut bertipe numerik dan modus untuk atribut bertipe kategorikal.

3. **Visualisasi Data:** Kami menggunakan berbagai jenis visualisasi data, seperti lineplot, boxplot, histogram, pie chart, dan matriks korelasi (heatmap). Kami juga melakukan contoh penggunaan groupby dan query untuk menganalisis data.

4. **Variabel Dependan dan Independen:** Variabel dependen atau target yang digunakan adalah `HujanBesok`, sementara variabel independen yang digunakan melibatkan atribut seperti `Lokasi`, `TempMin`, `TempMax`, `CurahHujan`, `Evaporasi`, `SinarMatahari`, `KecepatanAngin`, `Kelembapan9am`, `Kelembapan3pm`, `Awan9am`, `Awan3pm`, dan `HujanHariIni`.

5. **Transformasi Data:** Kami mengubah variabel kategorikal menjadi numerik menggunakan library Label Encoder.

6. **Feature Selection:** Proses feature selection dilakukan dengan menggunakan library F regression, di mana kami memilih 12 atribut yang paling berpengaruh.

7. **Pembagian Data:** Data dibagi menjadi training dan testing dengan proporsi 70% untuk training data dan 30% untuk testing data.

8. **Model Pengujian:** Kami menggunakan model logistic regression dan model SVM untuk prediksi. Hasil evaluasi model menunjukkan bahwa keduanya memiliki akurasi sekitar 84%, dengan perbedaan dalam nilai presisi, recall, dan F1 score.

9. **Kesimpulan Model:** Meskipun nilai akurasi antara model logistic regression dan SVM tidak berbeda jauh, kami memilih model logistic regression untuk deployment karena memiliki nilai presisi dan F1 score yang lebih tinggi.